# পর্ব 2: সংঘবদ্ধ শিক্ষার পরিচয়

শেষ বিভাগে, আমরা পয়েন্টারটেন্সারগুলি সম্পর্কে শিখেছি, যা ডিপ লার্নিং সংরক্ষণের গোপনীয়তার জন্য আমাদের প্রয়োজনীয় অন্তর্নিহিত অবকাঠামো তৈরি করে। এই বিভাগে, আমরা কীভাবে গভীর শিক্ষার অ্যালগরিদম, ফেডারেটেড লার্নিং সংরক্ষণের জন্য আমাদের প্রথম গোপনীয়তা প্রয়োগ করতে এই বেসিক সরঞ্জামগুলি ব্যবহার করব তা দেখতে যাচ্ছি।

লেখক:
- Andrew Trask - Twitter: [@iamtrask](https://twitter.com/iamtrask)

অনুবাদক:
- Sayantan Das - Github: [@ucalyptus](https://github.com/ucalyptus)

### ফেডারেটড লার্নিং কী?

এটি ডিপ লার্নিং মডেলগুলি প্রশিক্ষণের একটি সহজ, শক্তিশালী উপায়। আপনি যদি প্রশিক্ষণের ডেটা সম্পর্কে চিন্তা করেন, এটি সর্বদা কোনও ধরণের সংগ্রহ প্রক্রিয়ার ফলাফল। লোকেরা (ডিভাইসের মাধ্যমে) বাস্তব বিশ্বে ইভেন্টগুলি রেকর্ড করে ডেটা উত্পন্ন করে। সাধারণত, এই ডেটাটি একক, কেন্দ্রীয় অবস্থানে সংযুক্ত করা হয় যাতে আপনি একটি মেশিন লার্নিং মডেলকে প্রশিক্ষণ দিতে পারেন। ফেডারেটেড লার্নিং এটিকে নিজের মাথায় ঘুরিয়ে দেয়!

মডেলটিতে প্রশিক্ষণ ডেটা আনার পরিবর্তে (একটি কেন্দ্রীয় সার্ভার) আপনি মডেলটিকে প্রশিক্ষণের ডেটাতে নিয়ে আসুন (এটি যেখানেই থাকুক না কেন)।

ধারণাটি হ'ল এটি যার দ্বারা ডেটা তৈরি করছে সে কেবলমাত্র স্থায়ী অনুলিপিটির মালিক হতে পারে এবং যার ফলে এটিতে কখন অ্যাক্সেস রয়েছে তা নিয়ন্ত্রণ বজায় রাখে। খুব সুন্দর, তাই না?

# বিভাগ 2.1 - একটি খেলনা সংযুক্ত শিক্ষার উদাহরণ

আসুন একটি খেলনা মডেলকে কেন্দ্রিক উপায়ে প্রশিক্ষণ দিয়ে শুরু করি। মডেলগুলি পাওয়ার সাথে সাথে এটি একটি সাধারণ। আমাদের প্রথম প্রয়োজন:

- একটি খেলনা ডেটাসেট
- একজন মডেল
- ডেটা ফিট করার জন্য একটি মডেল প্রশিক্ষণের জন্য কিছু প্রাথমিক প্রশিক্ষণের যুক্তি।

দ্রষ্টব্য: যদি এই এপিআইটি আপনার কাছে অপরিচিত হয় - তবে [ফাস্ট.ai](http://fast.ai) এ যান এবং এই টিউটোরিয়ালটি চালিয়ে যাওয়ার আগে তাদের কোর্সটি গ্রহণ করুন।

In [ ]:
import torch
from torch import nn
from torch import optim

In [ ]:

# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.]], requires_grad=True)

# A Toy Model
model = nn.Linear(2,1)

def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(20):

        # 1) erase previous gradients (if they exist)
        opt.zero_grad()

        # 2) make a prediction
        pred = model(data)

        # 3) calculate how much we missed
        loss = ((pred - target)**2).sum()

        # 4) figure out which weights caused us to miss
        loss.backward()

        # 5) change those weights
        opt.step()

        # 6) print our progress
        print(loss.data)

In [ ]:
train()

এবং সেখানে আপনি এটা আছে! আমরা প্রচলিত পদ্ধতিতে একটি বেসিক মডেলকে প্রশিক্ষণ দিয়েছি। আমাদের সমস্ত ডেটা আমাদের স্থানীয় মেশিনে একত্রিত হয় এবং আমরা এটি আমাদের মডেলটিতে আপডেট করতে ব্যবহার করতে পারি। ফেডারেটেড লার্নিং অবশ্য এইভাবে কাজ করে না। সুতরাং, ফেডারেট লার্নিংয়ের উপায়ে এটি করার জন্য এই উদাহরণটি সংশোধন করি!

সুতরাং, আমাদের কী দরকার:

- কয়েকজন শ্রমিক তৈরি করুন
- প্রতিটি কর্মীর উপর প্রশিক্ষণের ডেটা নির্দেশক পান
- সংঘবদ্ধ শিক্ষার জন্য প্রশিক্ষণের যুক্তি আপডেট করা

     নতুন প্রশিক্ষণের পদক্ষেপ:
     - কর্মী সংশোধন করতে মডেল প্রেরণ করুন
     - সেখানে অবস্থিত ডেটা উপর প্রশিক্ষণ
     - মডেলটি ফিরে পাবেন এবং পরবর্তী কর্মীদের সাথে পুনরাবৃত্তি করুন

In [ ]:
import syft as sy
hook = sy.TorchHook(torch)


In [ ]:
# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")


In [ ]:
# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.]], requires_grad=True)

# get pointers to training data on each worker by
# sending some training data to bob and alice
data_bob = data[0:2]
target_bob = target[0:2]

data_alice = data[2:]
target_alice = target[2:]

# Iniitalize A Toy Model
model = nn.Linear(2,1)

data_bob = data_bob.send(bob)
data_alice = data_alice.send(alice)
target_bob = target_bob.send(bob)
target_alice = target_alice.send(alice)

# organize pointers into a list
datasets = [(data_bob,target_bob),(data_alice,target_alice)]

opt = optim.SGD(params=model.parameters(),lr=0.1)

In [ ]:
def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(10):
        
        # NEW) iterate through each worker's dataset
        for data,target in datasets:
            
            # NEW) send model to correct worker
            model.send(data.location)

            # 1) erase previous gradients (if they exist)
            opt.zero_grad()

            # 2) make a prediction
            pred = model(data)

            # 3) calculate how much we missed
            loss = ((pred - target)**2).sum()

            # 4) figure out which weights caused us to miss
            loss.backward()

            # 5) change those weights
            opt.step()
            
            # NEW) get model (with gradients)
            model.get()

            # 6) print our progress
            print(loss.get()) # NEW) slight edit... need to call .get() on loss\
    
# federated averaging

In [ ]:
train()

## সাবাশ!

ও ভয়েল! ফেডারেটেড লার্নিং ব্যবহার করে আমরা এখন একটি খুব সাধারণ ডিপ লার্নিং মডেল প্রশিক্ষণ দিচ্ছি! আমরা প্রতিটি কর্মীকে মডেল প্রেরণ করি, একটি নতুন গ্রেডিয়েন্ট উত্পন্ন করি এবং তারপরে গ্রেডিয়েন্টটি আমাদের স্থানীয় সার্ভারে ফিরিয়ে আনি যেখানে আমরা আমাদের বিশ্বব্যাপী মডেলটি আপডেট করি। এই প্রক্রিয়াতে কখনই আমরা অন্তর্নিহিত প্রশিক্ষণের ডেটা দেখতে বা দেখার জন্য অনুরোধ করি না! আমরা বব এবং অ্যালিসের গোপনীয়তা সংরক্ষণ করি !!!

## এই উদাহরণের স্বল্পতা

সুতরাং, যদিও এই উদাহরণটি ফেডারেটড লার্নিংয়ের একটি দুর্দান্ত ভূমিকা, এটির এখনও কিছু বড় ত্রুটি রয়েছে। সর্বাধিক উল্লেখযোগ্যভাবে, যখন আমরা `Model.get () call কল করি এবং বব বা অ্যালিসের কাছ থেকে আপডেট হওয়া মডেলটি পাই, তখন আমরা বব এবং অ্যালিসের প্রশিক্ষণের ডেটা সম্পর্কে তাদের গ্রেডিয়েন্টগুলি দেখে আসলে অনেক কিছু জানতে পারি। কিছু ক্ষেত্রে, আমরা তাদের প্রশিক্ষণের ডেটা পুরোপুরি পুনরুদ্ধার করতে পারি!

তো, কী করার আছে? ঠিক আছে, লোকেদের প্রথম কৌশলটি নিয়োগ করা হয় ** কেন্দ্রীয় সার্ভারে আপলোড করার আগে একাধিক ব্যক্তির কাছে গ্রেডিয়েন্ট গড়ানো **। এই কৌশলটির জন্য পয়েন্টারটেন্সর অবজেক্টের আরও কিছু পরিশীলিত ব্যবহারের প্রয়োজন হবে। সুতরাং, পরবর্তী বিভাগে, আমরা আরও উন্নত পয়েন্টার কার্যকারিতা সম্পর্কে জানতে কিছুটা সময় নেব এবং তারপরে আমরা এই ফেডারেট লার্নিং উদাহরণটি আপগ্রেড করব

# অভিনন্দন !!! - সম্প্রদায় যোগদানের সময়!

এই নোটবুক টিউটোরিয়ালটি সম্পন্ন করার জন্য অভিনন্দন! আপনি যদি এটি উপভোগ করেন এবং গোপনীয়তা সংরক্ষণ, এআই এবং এআই সরবরাহ চেইনের (ডেটা) বিকেন্দ্রীভূত মালিকানার দিকে আন্দোলনে যোগ দিতে চান, আপনি নিম্নলিখিত উপায়ে এটি করতে পারেন!

### গিটহাবে স্টার পাইসাইফ্ট

আমাদের সম্প্রদায়কে সাহায্য করার সবচেয়ে সহজ উপায় হ'ল রেপোসকে অভিনীত করা! এটি আমরা যে শীতল সরঞ্জামগুলি তৈরি করছি তার সচেতনতা বাড়াতে সহায়তা করে।

- [স্টার পাইসাইফ্ট](https://github.com/OpenMined/PySyft)

### আমাদের স্ল্যাচে যোগ দিন!

সর্বশেষতম অগ্রগতিতে আপ টু ডেট রাখার সর্বোত্তম উপায় হ'ল আমাদের সম্প্রদায়ে যোগদান করা! আপনি [http://slack.openmined.org](http://slack.openmined.org) এ ফর্মটি পূরণ করে এটি করতে পারেন

### একটি কোড প্রকল্পে যোগদান করুন!

আমাদের সম্প্রদায়ে অবদান রাখার সর্বোত্তম উপায় হ'ল কোড অবদানকারী হয়ে উঠুন! যে কোনও সময় আপনি পাইসাইফ্ট গিটহাব ইস্যু পৃষ্ঠাতে যেতে পারেন এবং "প্রকল্পগুলি" এর জন্য ফিল্টার করতে পারেন। এটি আপনাকে শীর্ষ স্তরের সমস্ত টিকিট দেখিয়ে দেবে যে আপনি কোন প্রকল্পগুলিতে যোগদান করতে পারেন তার একটি ওভারভিউ দেয়! আপনি যদি কোনও প্রকল্পে যোগ দিতে না চান তবে আপনি কিছুটা কোডিং করতে চান, আপনি আরও ভাল "ওয়ান অফ" মিনি-প্রকল্পগুলি গিটহাব ইস্যুগুলি "ভাল প্রথম ইস্যু" চিহ্নিত করে অনুসন্ধান করতে পারেন।

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)
### দান করুন

আপনার যদি আমাদের কোডবেসে অবদান রাখার সময় না থাকে তবে তবুও সমর্থন leণ দিতে চান, আপনি আমাদের ওপেন কালেক্টিভেরও ব্যাকের হয়ে উঠতে পারেন। সমস্ত অনুদান আমাদের ওয়েব হোস্টিং এবং অন্যান্য সম্প্রদায় ব্যয় যেমন হ্যাকাথনস এবং মেটআপগুলির দিকে যায়!

[ওপেনমাইন্ডের মুক্ত সমাহারক পৃষ্ঠা](https://opencollective.com/openmined)